In [5]:
import time
import numpy as np
from torch import nn, optim
from torchvision import models
from torchvision.transforms import Compose, ToTensor
from skorch import NeuralNetClassifier
from dlordinal.datasets import FGNet
from dlordinal.losses import TriangularLoss
from dlordinal.metrics import amae, mmae



In [9]:
# 1. 准备数据
fgnet_train = FGNet(
    root="./datasets",
    train=True,
    #强制转换为 int64）
    target_transform=lambda x: np.array(x, dtype=np.int64),
    transform=Compose([ToTensor()]),
)


fgnet_test = FGNet(
    root="./datasets",
    train=False,
    #强制转换为 int64）
    target_transform=lambda x: np.array(x, dtype=np.int64),
    transform=Compose([ToTensor()]),
)

num_classes_fgnet = len(fgnet_train.classes)

# 2. 定义模型
model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, num_classes_fgnet)

# 3. 定义损失函数
loss_fn = TriangularLoss(base_loss=nn.CrossEntropyLoss(), num_classes=num_classes_fgnet)

# 4. 定义 Skorch 估计器 (修改了这里)
estimator = NeuralNetClassifier(
    module=model,
    criterion=loss_fn,
    optimizer=optim.Adam,   # 显式使用 optim.Adam
    lr=1e-3,
    max_epochs=25,
    batch_size=128,         # 论文中提到 Batch size 为 128 
    
    # ---开启详细日志 ---
    verbose=1,              # 1=打印表格, 0=静默。设置为 1 可以实时看到每个 epoch 的 loss
    iterator_train__shuffle=True, # 训练时打乱数据通常是必须的
)

print("="*30)
print(f"开始训练... 配置参数: LR={estimator.lr}, Epochs={estimator.max_epochs}")
print("="*30)

# --- 增加计时器 ---
start_time = time.time() 

# 开始训练
estimator.fit(X=fgnet_train, y=fgnet_train.targets)

end_time = time.time()
total_time = end_time - start_time

# --- 打印统计信息 ---
print("\n" + "="*30)
print(f"训练完成！")
print(f"总耗时: {total_time:.2f} 秒")
# 获取最后一个 epoch 的训练记录
final_history = estimator.history[-1]
print(f"最终训练 Loss: {final_history['train_loss']:.4f}")
print(f"最终验证 Loss: {final_history['valid_loss']:.4f}")
print("="*30 + "\n")

# 5. 预测与评估
train_probs = estimator.predict_proba(fgnet_train)
test_probs = estimator.predict_proba(fgnet_test)

# Metrics
amae_metric = amae(np.array(fgnet_test.targets), test_probs)
mmae_metric = mmae(np.array(fgnet_test.targets), test_probs)
print(f"Test AMAE: {amae_metric}, Test MMAE: {mmae_metric}")

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Files already downloaded and verified
Files already processed and verified
Files already split and verified
开始训练... 配置参数: LR=0.001, Epochs=25
  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        1.7511       0.4286        1.7567  29.4165
      2        0.9048       0.2733        2.4967  23.9303
      3        0.5728       0.3913        2.0083  25.3854
      4        0.4765       0.5155        1.7462  23.4631
      5        0.4312       0.5466        1.7226  23.1073
      6        0.4053       0.5528        1.4572  24.2548
      7        0.3872       0.6149        1.2848  24.6281
      8        0.3750       0.6211        1.1680  22.9204
      9        0.3660       0.6087        1.1128  23.4689
     10        0.3596       0.6087        1.1521  22.6132
     11        0.3582       0.5901        1.0931  24.2